## Multiple linear regression
<br><u>
#### Background and Intuition</u>
This notebook presents an example of machine learning using multiple linear regression. In  multiple linear regression, we use data with several independent variable ($x_1, ... , x_n$). The goal is to fit the data to the linear form:
    $$y_i = \beta_0 1 + \beta_1 x_{i1} + \cdots + \beta_p x_{ip} + \varepsilon_i = \mathbf{x}^T_i\mathbf{\beta} + \varepsilon_i,
 \qquad i = 1, \ldots n$$
or more conveniently: $$\mathbf{y} = \mathbf{X} \mathbf\beta + \mathbf\varepsilon$$
 
 Where $\mathbf\beta$ is a $(p+1)$-dimensional parameter vector, and $\beta _{0}$ is a constant (offset) term. Elements of this vector are interpreted as partial derivatives of $\mathbf{y}$ with respect to the corresponding independent variables. $\mathbf\varepsilon$ is a vector of residuals of the data from the proposed model. 
 
 The most common method for calculating the unknown parameters $\mathbf\beta$ of the model is the Ordinary Least Squares method. OLS minimizes the sum of squared residuals, leading to the closed form equation: $${\hat {\boldsymbol {\beta }}}=(\mathbf {X} ^{\top }\mathbf {X} )^{-1}\mathbf {X} ^{\top }\mathbf {y} =\left(\sum \mathbf {x} _{i}\mathbf {x} _{i}^{\top }\right)^{-1}\left(\sum \mathbf {x} _{i}y_{i}\right).$$

In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm

In [2]:
# read the dataset
data = pd.read_csv('50_Startups.csv')
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
# selecting features and target 
columns = data.columns
target = data['Profit']
features = data[columns.drop('Profit')]
features.head()

,R&D Spend,Administration,Marketing Spend,State
0,165349.20,136897.80,471784.10,New York
1,162597.70,151377.59,443898.53,California
2,153441.51,101145.55,407934.54,Florida
3,144372.41,118671.85,383199.62,New York
4,142107.34,91391.77,366168.42,Florida


In [4]:
# encode categorical column to numeric values, drop original column
features  = pd.concat([features, pd.get_dummies(features['State'])], axis=1)
features = features.drop('State', axis=1)
features.head()

,R&D Spend,Administration,Marketing Spend,California,Florida,New York
0,165349.20,136897.80,471784.10,0,0,1
1,162597.70,151377.59,443898.53,1,0,0
2,153441.51,101145.55,407934.54,0,1,0
3,144372.41,118671.85,383199.62,0,0,1
4,142107.34,91391.77,366168.42,0,1,0


In [5]:
# since any state column entry can be infered from the other two, drop redundant data 
features = features.drop('California', axis=1)
features.head()
# Note: sklearn does this automatically, but it is good practice for using other software

,R&D Spend,Administration,Marketing Spend,Florida,New York
0,165349.20,136897.80,471784.10,0,1
1,162597.70,151377.59,443898.53,0,0
2,153441.51,101145.55,407934.54,1,0
3,144372.41,118671.85,383199.62,0,1
4,142107.34,91391.77,366168.42,1,0


In [6]:
# split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)
# Note: sklearn takes care of variable scaling automatically

In [7]:
# instantiate and fit linear regression model to training set
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
# generate predictions for the test set
predictions = lr.predict(X_test)
predictions

array([ 103015.20159796,  132582.27760816,  132447.73845174,
         71976.09851258,  178537.48221055,  116161.24230165,
         67851.69209676,   98791.73374687,  113969.43533012,
        167921.0656955 ])

In [9]:
y_test

28    103282.38
11    144259.40
10    146121.95
41     77798.83
2     191050.39
27    105008.31
38     81229.06
31     97483.56
22    110352.25
4     166187.94
Name: Profit, dtype: float64

#### Feature Selection
To improve the model, we should drop certain columns depending on how they affect the results. Since the statsmodels api provides more useful information than sklearn, we will use it to aid in feature selection.

Note that sklearn automatically adds a column of 1's before applying regression algorithms, but statsmodels does not. This column of 1's takes the place of constant vector $\beta_0$ in the model.

<u>Process for eliminating features:</u><br>
1.) Select a cutoff significance level to decide when to stop removing columns. 5% is a common value.<br>
2.) Train a model with the full feature matrix.<br>
3.) If any feature has a p-value above the cutoff, remove the column with the highest p-value.<br>
4.) Retrain the model and check new p-values. Repeat until cutoff value is achieved.

In [10]:
# add a column of 1's to the feature matrix
features['x_0']  = np.ones((data.shape[0], 1), dtype=int)
features.head()

,R&D Spend,Administration,Marketing Spend,Florida,New York,x_0
0,165349.20,136897.80,471784.10,0,1,1
1,162597.70,151377.59,443898.53,0,0,1
2,153441.51,101145.55,407934.54,1,0,1
3,144372.41,118671.85,383199.62,0,1,1
4,142107.34,91391.77,366168.42,1,0,1


In [11]:
# create a new holder for chosen features
optimal_features = features
optimal_features.head()

,R&D Spend,Administration,Marketing Spend,Florida,New York,x_0
0,165349.20,136897.80,471784.10,0,1,1
1,162597.70,151377.59,443898.53,0,0,1
2,153441.51,101145.55,407934.54,1,0,1
3,144372.41,118671.85,383199.62,0,1,1
4,142107.34,91391.77,366168.42,1,0,1


In [12]:
OLS_regressor = sm.OLS(endog=target, exog=optimal_features).fit()
OLS_regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Fri, 20 Oct 2017   Prob (F-statistic):           1.34e-27
Time:                        13:21:41   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
R&D Spend           0.8060      0.046     17.369      0.000       0.712       0.900
Administration     -0.0270      0.052     -0.517      0.608      -0.132       0.078
Marketing Spend     0.0270      0.017      1.574      0.123      -0.008       0.062
Florida           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
New York          -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
x_0              5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
# begin removing columns with the highest p-values
optimal_features = optimal_features.drop('New York', axis=1)
optimal_features.head()

,R&D Spend,Administration,Marketing Spend,Florida,x_0
0,165349.20,136897.80,471784.10,0,1
1,162597.70,151377.59,443898.53,0,1
2,153441.51,101145.55,407934.54,1,1
3,144372.41,118671.85,383199.62,0,1
4,142107.34,91391.77,366168.42,1,1


In [14]:
# retrain the model excluding the 'New York' column
OLS_regressor = sm.OLS(endog=target, exog=optimal_features).fit()
OLS_regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Fri, 20 Oct 2017   Prob (F-statistic):           8.49e-29
Time:                        13:22:01   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
R&D Spend           0.8060      0.046     17.606      0.000       0.714       0.898
Administration     -0.0270      0.052     -0.523      0.604      -0.131       0.077
Marketing Spend     0.0270      0.017      1.592      0.118      -0.007       0.061
Florida           220.1585   2900.536      0.076      0.940   -5621.821    6062.138
x_0              5.011e+04   6647.870      7.537      0.000    3.67e+04    6.35e+04
==============================================================================
Omnibus:                       14.758   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.172
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
# remove the next columns with the highest p-value and retrain
optimal_features = optimal_features.drop('Florida', axis=1)
OLS_regressor = sm.OLS(endog=target, exog=optimal_features).fit()
OLS_regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Fri, 20 Oct 2017   Prob (F-statistic):           4.53e-30
Time:                        13:22:11   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
R&D Spend           0.8057      0.045     17.846      0.000       0.715       0.897
Administration     -0.0268      0.051     -0.526      0.602      -0.130       0.076
Marketing Spend     0.0272      0.016      1.655      0.105      -0.006       0.060
x_0              5.012e+04   6572.353      7.626      0.000    3.69e+04    6.34e+04
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
# remove the next columns with the highest p-value and retrain
optimal_features = optimal_features.drop('Administration', axis=1)
OLS_regressor = sm.OLS(endog=target, exog=optimal_features).fit()
OLS_regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Fri, 20 Oct 2017   Prob (F-statistic):           2.16e-31
Time:                        13:23:39   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
R&D Spend           0.7966      0.041     19.266      0.000       0.713       0.880
Marketing Spend     0.0299      0.016      1.927      0.060      -0.001       0.061
x_0              4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Evaluating the model's performance
At this point, we can decide whether or not to drop Marketing Spend from the model. It's p-value is .06, near the .05 cutoff originally specified. R-squared and Adjusted R-squared are metrics used to decide which features optimally tune the model.

Intuitively, the $R^2$ metric is a measure of how close the "best fit" parameters generated by the modelare to parameters given by the average of all the points.  The closer the model is to the average, the less information (and less usefulness) the model provides. Afterall, we can computer the average much more easily than a regression model.

#### <u>Calculating R-squared and Adjusted R-squared</u>

The total sum of squares is given by $SS_{\text{tot}}=\sum _{i}(y_{i}-{\bar {y}})^{2}$. The sum of squared residuals is ${\displaystyle SS_{\text{res}}=\sum _{i}(y_{i}-f_{i})^{2}=\sum _{i}e_{i}^{2}\,}$ where $f_i$ is the predicted value associated with that data point.

Finally we calculate $ R^{2} = 1-{SS_{\rm {res}} \over SS_{\rm {tot}}}.\ $

$R^2$ closer to 1 indicates that the model performs well. Lower values indicate worse performance.

$R^2$ can become less meaningful as the number of independent variables increases. Adjusted $R^2$ is an attempt to correct for this:
$${\bar {R}}^{2}={1-(1-R^{2}){n-1 \over n-p-1}}={R^{2}-(1-R^{2}){p \over n-p-1}}$$
where $p$ is the number of independent variables, $n$ is the number of data points.

Checking adjusted $R^2$ is a handy way to decide if making changes to the model has a positive or negative effect overall.

In [17]:
optimal_features2 = optimal_features.drop('Marketing Spend', axis=1)
OLS_regressor = sm.OLS(endog=target, exog=optimal_features2).fit()
OLS_regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Fri, 20 Oct 2017   Prob (F-statistic):           3.50e-32
Time:                        14:34:34   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
R&D Spend      0.8543      0.029     29.151      0.000       0.795       0.913
x_0         4.903e+04   2537.897     19.320      0.000    4.39e+04    5.41e+04
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.536
Skew:                          -0.911   Prob(JB):                     9.44e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

After removing the 'Market Spend' column from the model, adjusted-$R^2$ fell, so the best model includes Marketing Spend and R&D Spend.